# Dataset Preparation Tutorial

Welcome to the dataset preparation tutorial! In this notebook, we will download the toy data set for the tutorial and prepare the necessary tables used for later analysis. Here are the steps we will review:

1. Verify prerequisites
2. Create a new project workspace
3. Review sample dataset
4. Build the proxy table
5. Run regional annotation ETL

**NOTE**: All of the configuration files for this tutorial have been provided in the container. The host and port values in the configuration files are dynamically set based on your system. 

**NOTE**: The current working directory is '~/vmount/notebooks'. All file and directory paths specified in the configuration files are relative to the current working directory. 

## 1. Verify prerequisites

Here are the software prerequisites for executing tasks with luna packages. These prerequiristes have already been baked into this docker container. Too view the setup, please see the corresponding dockerfile. 

In [ ]:
!python3 --version
!java -version
!echo LUNA_HOME: $LUNA_HOME
!which jupyter
!pip list | grep luna-
import luna
luna.__path__
import luna.pathology
luna.pathology.__path__

## 2. Create a new project workspace



Next, we create a luna home space and place the configuration files there. Using a manifest file, we will create a project workspace for your configurations, data, models, and outputs to go for this tutorial.

In [ ]:
%%bash
mkdir -p ~/luna
cp -R ~/vmount/conf ~/luna
cat ~/luna/conf/manifest.yaml
python3 -m luna.project.generate --manifest_file ~/luna/conf/manifest.yaml
tree ~/vmount/PRO_12-123

You should now see a new directory called *PRO_12-123* with the manifest file in it. This will be your project workspace!

## 3. Review sample dataset

The data that you will be using for this tutorial is a set of 5 whole slide images of ovarian cancer H&E slides, available in the svs file format. Whole slide imaging refers to the scanning of conventional glass slides for research purposes; in this case, these are slides that oncologists have used to inspecting cancer samples!

While bringing up the DSA container, we already ran a script to get the data, and set up DSA. The `vmount/provision.py` script ran these steps:
  
  - Set up admin user and default assetstore
  
  - Download sample data from [public kitware site](https://data.kitware.com/#user/61b9f3dc4acac99f42ca7678/folder/61b9f4564acac99f42ca7692). to `~/vmount/PRO_12-123/data/toy_data_set/`
  
  - Create a collection and add slides/annotations to your local DSA


In [ ]:
%%bash
tree ~/vmount/PRO_12-123/data/toy_data_set

If you want to import your own data, you can do so from your local filesystem as well as an object store. For more details, refer to the [girder user documentation](https://girder.readthedocs.io/en/latest/user-guide.html#assetstores)

To import images from your local filesystem, 

- Login to DSA with admin/password
- Add images to your local computer at `vmount/assetstore` 
- Navigate to **Admin Console** -> **Assetstores**
- From the default assetstore, click on **Import data**
- Specify the path to the images you wish to import. e.g. `/assetstore/yourimage` and click import

As the `/assetstore` mount is available to DSA, this import should be much faster than uploading the image through the **Upload files** in the UI.


## 4. Build the proxy table

Now, we will run the Whole Slide Image (WSI) ETL to build a meta-data catalog of the slides in a proxy table. 

For reference, ETL stands for extract-transform-load; it is the method that often involves cleaning data, transforming data types, and loading data into different systems. 

In [ ]:
!slide_etl ~/vmount/PRO_12-123/data/toy_data_set \
--project_name PRO_12-123 --comment "Example Ingestion Job" \
--store_url "" --no-write \
-o ../PRO_12-123/data/toy_data_set/table/SLIDES 

This step may take a while. At the end, your proxy table should be generated!

Before we view the table, we must first update it to associate patient ID's with the slides. This is necessary for correctly training and validating the machine learning model in the coming notebooks. Once the slides are divided into "tiles" in the next notebook, the tiles are split between the training and validation sets for the ML model. If the tiles do not have patient ID's associated with them, then it is possible for tiles from one individual to appear in both the training and validation of the model; this would cause researchers to have an exaggerated interpretation of the model's accuracy, since we would essentially be validating the model on information that is too near to what it has already seen. 

Note that we will not be using patient IDs associated with MSK. Instead, we will be using spoof IDs that will suffice for this tutorial. When running this workflow with real data, make sure to include the IDs safely and securely. Run the following block of code to add a 'patient_id' column to the table and store it using Spark.

Next, we may view the WSI table! This table should have the metadata associated with the WSI slides that you just collected, including the patient IDs. 

In [ ]:
import pandas as pd
pd.read_parquet("../PRO_12-123/data/toy_data_set/table/SLIDES/slide_ingest_PRO_12-123.parquet")

If the table is depicted above, congratulations, you  have successfully run the Whole Slide Image (WSI) ETL to database the slides!

## Run the regional annotation ETL

The whole slide images that you downloaded are images of ovarian cancer, but not every pixel on each slide is a tumor. In fact, the images show tumor cells, normal ovarian cells and more. A non-expert annotated this slide for demo purposes only.

The regional annotation ETL performs the following steps

- Downloads DSA json annotations
- Converts DSA jsons to GeoJSON format, which is compatible with downstream applications
- Saves configs in your `~/vmount/PRO_12-123/configs/REGIONAL_METADATA_RESULTS`
- Saves parquet table in your `~/vmount/PRO_12-123/tables/REGIONAL_METADATA_RESULTS `


To run the regional annotation ETL, we use the `dsa_annotation` CLI. For more details on the dsa_annotation, and the annotations we support, please checkout the `7_dsa-annotation.ipynb` notebook.

**Note**: details of your DSA instance is specified as `DSA_URI` in `../conf/dsa_regional_annotation.yaml` and should be updated to reflect your DSA setup.  If you are using the docker, replace the `localhost` with the IP you get from running:

```docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' luna_tutorial_girder_1```


In [ ]:
!dsa_annotation http://pllimsksparky3:8081/api/v1 \
    --collection_name "TCGA collection" \
    --annotation_name "ov_regional" \
    --username admin --password password \
    --output_dir ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS 

To check that the regional annotation ETL was correctly run, after the Jupyter cell finishes, you may load the regional annotations table! This table contains the metadata saved from running the ETL. It includes paths to the bitmap files, numpy files, and geoJSON files that were mentioned before. To load the table, run the following code cell: 

In [ ]:
from pyarrow.parquet import read_table

pd.read_parquet("../PRO_12-123/data/toy_data_set/table/ANNOTATIONS/slide_annotation_dataset_TCGA collection_ov_regional.parquet")

Last, lets get our geojsons and join on slide id!

In [ ]:
pd.read_parquet("../PRO_12-123/data/toy_data_set/table/ANNOTATIONS/slide_annotation_dataset_TCGA collection_ov_regional.parquet") \
    .query("type=='geojson'")[['slide_geojson']] \
    .join(
        pd.read_parquet("../PRO_12-123/data/toy_data_set/table/SLIDES/slide_ingest_PRO_12-123.parquet")['slide_image']
    )


At this point, you have successfully set up your workspace, dowloaded the data, and run both the pathology and regional annotation ETLs to prepare your data. You are ready to move on to the tiling notebook!